In [2]:
import configparser
from pinecone import Pinecone,ServerlessSpec
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone_text.sparse import BM25Encoder
from langchain import LLMChain, PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Pinecone
from langchain_groq import ChatGroq
from faster_whisper import WhisperModel
import warnings
warnings.filterwarnings("ignore")

/home/growlt240/Documents/VoiceImpersonationPOC-01/poc1/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
# Function to connect to Pinecone DB
def connect_to_pinecone(api_key,index_name):   
    # Initialize pinecone client
    pc =Pinecone(api_key=api_key)
    # # Listing the existing indices
    # pc.list_indexes().names()

    # create index if does not exist
    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=384, # dimension of dense vector
            metric='dotproduct',# dotproduct sparse matrix 
            spec=ServerlessSpec(cloud='aws',region="us-east-1")    
        ) 
    # storing the index name for further usage
    index = pc.Index(index_name)
    return index

In [4]:
# Function to retrieve context using Pinecone
def context_retriever(user_query,index, k=7):

    # Load the model for embeddings
    embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

    # Load the model for sparse matrix encoding
    bm25_encoder =BM25Encoder().default()

    # Using Hybrid Search Reriever for context extraction
    retriever = PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index, top_k= k)

    context = retriever.invoke(user_query)

    return context

In [5]:
# help(PineconeHybridSearchRetriever)

In [6]:
# ! pip install faster-whisper


In [7]:
# Function to process audio query
def process_audio(audio_file_path,model):
    segments, info = model.transcribe(audio_file_path)
    # Combine the text from all segments
    text = " ".join(segment.text for segment in segments)
    return text
 
# # Example usage
# audio_file_path = "/home/growlt240/Documents/VoiceImpersonationPOC-01/Conference.wav"
# transcribed_text = process_audio(audio_file_path)
# print(transcribed_text)
 


In [8]:
# Function to generate LLM response
def generate_llm_response(context, question, api_key, model_name):
    # Use LLaMA 3 for response generation
    llm = ChatGroq(groq_api_key=api_key, model_name= model_name, temperature= 0.2)
 
    # Define the prompt template using PromptTemplate
    prompt_template = PromptTemplate(
        template="""
        You are a helpful and concise assistant. Provide detailed and accurate responses based on the given context.
        If the answer is not available in the context, clearly state: "The information you requested is not available in the provided context."
 
        Context: {context}
 
        Question: {question}
 
        Answer:
        """,
        input_variables=["context", "question"]
    )
 
    # Chain for RAG (Retrieval-Augmented Generation)
    rag_chain = LLMChain(
        llm=llm,
        prompt=prompt_template
    )
 
    if not context:
        # Handle case where context is empty
        formatted_context = "No context available."
    else:
        formatted_context = "\n".join(str(item) for item in context)
    
    # # Define a maximum context length (e.g., 2000 characters or tokens)
    # MAX_CONTEXT_LENGTH = 2000
    
    # # Truncate context if necessary
    # if len(formatted_context) > MAX_CONTEXT_LENGTH:
    #     formatted_context = formatted_context[:MAX_CONTEXT_LENGTH]
    # Use 'run' instead of 'call'
    response = rag_chain.run(context=formatted_context, question=question)
    return response

In [9]:
def main(user_query):
    # Read the config file
    config = configparser.ConfigParser()
    config.read('config.ini')

    # Get the API details
    api_config = config['api']
    pinecone_api_key = api_config['pinecone_api_key']
    groq_api_key = api_config['groq_api_key']

    # required index
    index_name = "knowledge-base-growexx"
    llm_model = 'gemma2-9b-it'

    # Initialize Whisper model
    model_size = "distil-large-v2"
    model = WhisperModel(model_size, device="cpu", compute_type="float32")

    # user_query = input("Enter your question: ")

    #Check if user_query is audio or text
    if user_query.endswith('.wav'):
        text_query = process_audio(user_query, model)
    else:
        text_query = user_query
    
    # Connect to knowledge base 
    index = connect_to_pinecone(pinecone_api_key,index_name)
    
    # Retrieve context
    content = context_retriever(text_query,index)
    context =[]
    for i in content:
        context.append(i.page_content)
    
    # Generate response using LLM
    response = generate_llm_response(context, text_query, groq_api_key, llm_model)
    return response
    

In [10]:
user_query = input("Enter your question: ")
print(f"Response generated:/n{main(user_query)}")

Response generated:/nThe provided text does not contain any direct quotes or statements from clients about GrowExx. 


Therefore:  The information you requested is not available in the provided context. 



In [11]:
# audio input
print(main("/home/growlt240/Documents/VoiceImpersonationPOC-01/Record (online-voice-recorder.com).wav"))
#print(answer_query('what services growexx offers?'))

The provided text lists these services offered by GrowExx:

* Competitive .NET development services
* Business intelligence solutions
* AI solutions tailored to business objectives
* End-to-end Tableau consulting services
* DevOps consulting services 


Let me know if you have any other questions. 



In [12]:
# if __name__ == "__main__":
#     main()